# News Headline Generation

## Part 1: Data Preparation

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from keras.applications.densenet import preprocess_input, decode_predictions
from keras.utils.np_utils import to_categorical 



In [5]:
df2 = pd.read_csv("../all-the-news-2-1.csv")
sources = df2[['year', 'title', 'article', 'publication']]
sources.head()

,year,title,article,publication
0,2016,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",Vox
1,2016,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,Business Insider
2,2018,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",Reuters
3,2019,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,Reuters
4,2016,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,TMZ


### selected news sources
all at least 20,000 publications

<b>Everything</b>
- Fox News (right - 20,144)
- Reuters (center - 840,094)
- CNN (left - 127,602)

<b>Entertainment News</b>
- TMZ (49,595)
- Refinery29 (111,433)

<b>Business</b>
- Business Insider (57,953)